# Homework 03 - Alessandro Oddi
## Immagine di riferimento

![riferimento](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/stairSample.jpg?raw=true)

## Procedimento 

Scelta una scala di riferimento, ho iniziato creando il metodo richiesto dalla traccia che prendesse come parametri le dimensioni ***dx, dy, dz*** entro cui costruire la scala.
In particolare ho voluto fare particolare attenzione alla possibilità di avere una scala a singola o doppia rampa: scelta in base alla dimensione dy passata come parametro.

Ho seguito le direttive di [questo sito](http://www.uncome.it/casa/articolo/come-calcolare-una-scala-341.html) su come generare una scala a norma, rispettando la **legge di Blondel**. Con i primi calcoli del mio metodo sono riuscito ad estrapolare il numero minimo di gradini pedata e alzata, informazioni di fondamentale importanza per generare l'HPC della scala.

Con un metodo ausiliare poi mi sono occupato della creazione della scala vera e propria. Il metodo cicla sull'altezza dz raggiunta dopo l'aggiunta di ogni scalino. In caso di insufficente spazio sul piano dy viene generata una piattaforma e viene ruotato il piano di riferimento; tutto ciò procede finche non si è raggiunta la dimensione del parametro dz desiderata. A questo punto vine generata l'immagine HPC che viene ritornata al metodo principale.

In [1]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.005809 seconds


In [2]:
""" intRGBColor

Metodo preso dal web per ottenere colori da RGB
@param values: lista di tre interi RGB
@return: Colore
"""
def intRGBColor(values):
        return Color4f([ values[0]/255.0,
                         values[1]/255.0,
                         values[2]/255.0,
                         1.0])
BRICK = intRGBColor([211,153,127])
LIGHT_BROWN = intRGBColor([77,65,27])

In [3]:
""" ggpl_scale_a_rampa

Presi 3 valori in metri restituisce una scala a Rampa
@param dx: larghezza su x del gradino
@param dy: spazio che la scala occupa sul piano y
@param dz: spazio che la scala occupa sul piano z
@return: HPC della Scala
"""
def ggpl_scale_a_rampa(dx,dy,dz):
    dx=dx/2
    step = math.floor(dz * 100 / 18) + 1
    riser = dz * 100 / step
    tread = 64 - 2 * riser
    return buildStair(int(step),riser/100,tread/100,dx,dy,dz)
    

In [156]:
""" buildStair

Metodo ausiliare di ggpl_scale a rampa che si occupa di creare l'hpc della scala
@param step: numero di gradini minimo per coprire l'altezza dz
@param riser: alzata del gradino
@param tread: pedata del gradino
@param dx: larghezza su x del gradino
@param dy: spazio che la scala occupa sul piano y
@param dz: spazio che la scala occupa sul piano z
@return: HPC della Scala
"""
def buildStair(step,riser,tread,dx,dy,dz):
    
    hasPlatform=False
    maxStepB4Platform=0
    dzCurrent=0
    y=[riser]
    listStair=[]
    listStair.append(T(2)(tread*3))
    
    
    """Si genera l'oggetto che rappresenta il singolo gradino"""
    firstStair = MKPOL([[[tread,riser],[tread,0],[0,0],[0,riser]],[[1,2,3,4]],1])
    firstStair=PROD([firstStair,QUOTE([dx])])
    firstStair=STRUCT([T([2,3])([tread,riser]),R([1, 3])(-math.pi / 2),R([1, 2])(3 * math.pi / 2),firstStair])
    singleStair = MKPOL([[[tread,riser],[tread,0],[tread,-riser],[0,-riser],[0,0]],[[1,2,3,4,5]],1])
    singleStair=PROD([singleStair,QUOTE([dx])])
    singleStair=STRUCT([T([2,3])([tread,riser]),R([1, 3])(-math.pi / 2),R([1, 2])(3 * math.pi / 2),singleStair])
    x=[0.15*riser]
    nosing=CUBOID([dx,tread+0.1*tread])
    nosing=COLOR(LIGHT_BROWN)(PROD([nosing,QUOTE(x)]))
    handrail=MKPOL([[[0,0],[tread,riser],[tread,riser+riser*0.5],[0,riser-riser*0.5],[0,0]],[[1,2,3,4,5]],1])
    handrail=COLOR(LIGHT_BROWN)(PROD([handrail,QUOTE([0.08])]))
    supportHandrail=STRUCT([T(1)((0.01*3)),T(2)((tread+0.1*tread)/2-(0.02*5)),T(3)(riser*2),CUBOID([0.01,0.02,0.9]),T(2)(0.2*tread),CUBOID([0.01,0.02,0.95]),T(2)(0.2*tread),CUBOID([0.01,0.02,1])])
    firstStair=STRUCT([firstStair,T(3)(riser),T(2)(-0.1*tread),nosing,T(3)(0.8)])
    singleStair=STRUCT([T(2)(0.1*tread),supportHandrail,singleStair,T(3)(2*riser),T(2)(-0.1*tread),nosing,T(3)(0.8),R([1, 3])(-math.pi / 2),R([1, 2])( math.pi / 2),handrail,T(3)(-0.8)])
    listStair.append(T(2)(-tread))
    listStair.append(firstStair)
    listStair.append(T(2)(tread-0.1*tread))
    
    """ Viene creata la Piattarorma"""
    platform=CUBOID([dx*2+0.1,tread*3])
    platform=COLOR(LIGHT_BROWN)(PROD([platform,QUOTE(x)]))
    basePlatform=COLOR(WHITE)(PROD([CUBOID([dx*2+0.1,tread*3]),QUOTE(y)]))
    supportHandrail=STRUCT([CUBOID([0.01,0.02,0.95]),T(2)(0.2*tread),CUBOID([0.01,0.02,0.95]),T(2)(0.2*tread),CUBOID([0.01,0.02,0.95])])
    handrailPlatorform=COLOR(LIGHT_BROWN)(STRUCT([CUBOID([0.08,tread*3,riser-riser*0.5]),
                                          T([2])([tread*3-0.08]),CUBOID([dx*2+0.1,0.08,riser-riser*0.5]),
                                          T([1,2])([2*dx-0.08+0.1,-tread*3+0.08]),CUBOID([0.08,tread*3-0.08,riser-riser*0.5])]))
    frames= STRUCT([supportHandrail,T(2)((tread)),supportHandrail,
                                          T(2)((tread)),supportHandrail])
    supporthandrailPlatform=(STRUCT([T([1,2,3])([0.2*tread,0.3*tread,riser*0.15]),frames, T(2)(SIZE(2)(frames)+0.18*tread),
                                          R([2, 1])(math.pi / 2),T([2])(dx*0.2+0.05),supportHandrail,
                                          T([2])(dx*0.4),supportHandrail,T([2])(dx*0.35),supportHandrail,
                                          T([2])(dx*0.4),supportHandrail,T([1,2])([tread*0.3,dx*0.45+0.1]),
                                          R([2, 1])(math.pi / 2),frames]))
    platform=STRUCT([supporthandrailPlatform, platform,T(3)(-riser),basePlatform,T(3)(riser),T(3)(0.8+(riser-riser*0.5)*2),handrailPlatorform,T(3)(-0.8-(riser-riser*0.5)*2)])
    
    
    
    """Si controlla se il nuero minimo di scalini può entrare nello spazio a disposizione,
    in caso contrario si mette a True il fatto che la scala abbia una piattaforma """
    if((step>int(dy/tread))):
        hasPlatform=True
        maxStepB4Platform=int(dy/tread)-5 #max-Step-before-Platform
   
    """finche non viene raggiunta l'altezza parametrica si continuano ad agigungere gradini
    e in caso si raggiunga la dy massima consentita viene generata una piattaforma e creata
    una seconda rampa di gradini"""
    i=0
    while(dzCurrent<=dz):
        
        if(hasPlatform and (i+1)%maxStepB4Platform==0):
            listStair.append(T([2,3])([tread,riser*2]))
            listStair.append(platform)
            listStair.append(R([1, 2])( 3* math.pi))
            listStair.append(T([1,2,3])([-2*dx-0.1,-(tread+0.1*tread),-2*riser]))
        
        else:    
            if(i!=0):
                listStair.append(T([2,3])([tread,riser]))
            dzCurrent=dzCurrent+riser
            if(dzCurrent+riser*0.15<=dz):
                listStair.append(COLOR(WHITE)(singleStair))   
        
        i=i+1   
    
    listStair= STRUCT(listStair)
    if(hasPlatform):
        listStair= STRUCT([listStair,T([1,2])([dx,tread*3]),CUBOID([0.1,dy-tread*7,dz])])
    return listStair
   
            
        

In [ ]:
"""Alcuni esempi di scale generabili"""
VIEW(ggpl_scale_a_rampa(1.4,5.6,17.3))
VIEW(ggpl_scale_a_rampa(2,8,5))


## Immagini dei Risultati    

Scala a doppia rampa 1

![imag1](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test1.png?raw=true)

Scala a doppia rampa 2

![imag1](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test4.png?raw=true)

Scala a singola rampa

![imag1](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test3.png?raw=true)


Come si vede dall'immagine sottostante la scala generata occupa completamente lo spazio di un generico poligono di dimensioni dx,dy,dz

![imag1](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test2.png?raw=true)


## Aggiornamenti

Ho Aggiunto nell'ultimo commit anche il corrimano cosi che le scale assomiglino ancora di più alla scala presa come riferimento:

![esempioN](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test5.png?raw=true)

![esempioN](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test6.png?raw=true)

![esempioN](https://github.com/adixia/ggpl-LAB/blob/master/2016-10-28/test7.png?raw=true)
